In [5]:
import numpy as np
import os
from scipy.stats import spearmanr

# Paths to saved ISC vectors and HMM adjacency matrices
isc_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors'
adjacency_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/group_level_adjacency_matrices'

roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']
n_permutations = 5000

# Function to compute mean adjacency vector from HMM adjacency matrix
def compute_hmm_adjacency_vector(adjacency_matrix):
    return np.mean(adjacency_matrix, axis=1)

# Function to permute the HMM adjacency matrix by shuffling rows and columns
def permute_adjacency_matrix(adjacency_matrix):
    permuted_indices = np.random.permutation(adjacency_matrix.shape[0])
    permuted_matrix = adjacency_matrix[permuted_indices][:, permuted_indices]
    return permuted_matrix

for roi in roi_names:
    # Load the ISC vector for IS-RSA
    isc_path = os.path.join(isc_dir, f'{roi}_ISC_vector.npy')
    adjacency_path = os.path.join(adjacency_dir, f'{roi}_adjacency_matrix.npy')
    
    if os.path.exists(isc_path) and os.path.exists(adjacency_path):
        # Load ISC vector and HMM adjacency matrix
        isc_vector = np.load(isc_path)
        hmm_adjacency_matrix = np.load(adjacency_path)
        
        # Convert original HMM adjacency matrix to vector
        hmm_adjacency_vector = compute_hmm_adjacency_vector(hmm_adjacency_matrix)
        
        # Ensure both vectors have the same length
        if len(isc_vector) != len(hmm_adjacency_vector):
            print(f"Length mismatch for {roi}: ISC vector length is {len(isc_vector)}, "
                  f"HMM adjacency vector length is {len(hmm_adjacency_vector)}. Skipping.")
            continue
        
        # Compute observed Spearman correlation
        observed_correlation, _ = spearmanr(isc_vector, hmm_adjacency_vector)
        print(f"Observed Spearman correlation for {roi}: {observed_correlation}")
        
        # Permutation testing
        permuted_correlations = []
        for _ in range(n_permutations):
            # Permute the HMM adjacency matrix
            permuted_matrix = permute_adjacency_matrix(hmm_adjacency_matrix)
            # Compute the mean adjacency vector for the permuted matrix
            permuted_vector = compute_hmm_adjacency_vector(permuted_matrix)
            # Compute correlation with the ISC vector
            permuted_correlation, _ = spearmanr(isc_vector, permuted_vector)
            permuted_correlations.append(permuted_correlation)
        
        # Calculate p-value
        permuted_correlations = np.array(permuted_correlations)
        p_value = (np.sum(permuted_correlations >= observed_correlation) + 1) / (n_permutations + 1)
        
        print(f"Permutation test p-value for {roi}: {p_value}")
    else:
        print(f"ISC vector or HMM adjacency matrix for {roi} not found.")


Observed Spearman correlation for PTL: -0.06978747303321318
Permutation test p-value for PTL: 0.9926014797040592
Observed Spearman correlation for ATL: -0.04262954396332749
Permutation test p-value for ATL: 0.9300139972005599
Observed Spearman correlation for AG: -0.10195307027494525
Permutation test p-value for AG: 1.0
Observed Spearman correlation for IFG: -0.022423469738240078
Permutation test p-value for IFG: 0.79124175164967
Observed Spearman correlation for MFG: -0.04147219749137273
Permutation test p-value for MFG: 0.936612677464507
Observed Spearman correlation for IFGorb: 0.0008173096658477549
Permutation test p-value for IFGorb: 0.4907018596280744


In [7]:
import numpy as np
import os
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

# Paths to saved ISC vectors and HMM adjacency matrices
isc_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors'
adjacency_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/group_level_adjacency_matrices'
figures_dir = '/Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations'
os.makedirs(figures_dir, exist_ok=True)

roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']
n_permutations = 5000

# Function to compute mean adjacency vector from HMM adjacency matrix
def compute_hmm_adjacency_vector(adjacency_matrix):
    return np.mean(adjacency_matrix, axis=1)

for roi in roi_names:
    # Load the ISC vector for IS-RSA
    isc_path = os.path.join(isc_dir, f'{roi}_ISC_vector.npy')
    adjacency_path = os.path.join(adjacency_dir, f'{roi}_adjacency_matrix.npy')
    
    if os.path.exists(isc_path) and os.path.exists(adjacency_path):
        # Load ISC vector and HMM adjacency matrix
        isc_vector = np.load(isc_path)
        hmm_adjacency_matrix = np.load(adjacency_path)
        
        # Convert HMM adjacency matrix to vector
        hmm_adjacency_vector = compute_hmm_adjacency_vector(hmm_adjacency_matrix)
        
        # Ensure both vectors have the same length
        if len(isc_vector) != len(hmm_adjacency_vector):
            print(f"Length mismatch for {roi}: ISC vector length is {len(isc_vector)}, "
                  f"HMM adjacency vector length is {len(hmm_adjacency_vector)}. Skipping.")
            continue
        
        # Compute observed Spearman correlation
        observed_correlation, _ = spearmanr(isc_vector, hmm_adjacency_vector)
        print(f"Observed Spearman correlation for {roi}: {observed_correlation}")
        
        # Permutation testing
        permuted_correlations = []
        for _ in range(n_permutations):
            np.random.shuffle(isc_vector)  # Shuffle ISC vector
            permuted_correlation, _ = spearmanr(isc_vector, hmm_adjacency_vector)
            permuted_correlations.append(permuted_correlation)
        
        # Calculate two-tailed p-value
        permuted_correlations = np.array(permuted_correlations)
        p_value = (np.sum(np.abs(permuted_correlations) >= np.abs(observed_correlation)) + 1) / (n_permutations + 1)
        
        print(f"Permutation test p-value for {roi}: {p_value}")
        
        # Plot histogram of permuted correlations
        plt.figure(figsize=(8, 6))
        plt.hist(permuted_correlations, bins=30, color='skyblue', edgecolor='black')
        plt.axvline(observed_correlation, color='red', linestyle='dashed', linewidth=2, label=f'Observed Correlation: {observed_correlation:.4f}')
        plt.axvline(np.percentile(permuted_correlations, 2.5), color='green', linestyle='dashed', linewidth=2, label='95% Significance Thresholds')
        plt.axvline(np.percentile(permuted_correlations, 97.5), color='green', linestyle='dashed', linewidth=2)
        plt.title(f'Permutation Test for {roi}')
        plt.xlabel('Spearman Correlation')
        plt.ylabel('Frequency')
        plt.legend()
        plt.tight_layout()
        
        # Save the plot
        plot_path = os.path.join(figures_dir, f'{roi}_permutation_test.png')
        plt.savefig(plot_path)
        plt.close()
        
        print(f"Saved permutation test plot for {roi} at {plot_path}")
    else:
        print(f"ISC vector or HMM adjacency matrix for {roi} not found.")


Observed Spearman correlation for PTL: -0.06978747303321318
Permutation test p-value for PTL: 0.01199760047990402
Saved permutation test plot for PTL at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/PTL_permutation_test.png
Observed Spearman correlation for ATL: -0.04262954396332749
Permutation test p-value for ATL: 0.1217756448710258
Saved permutation test plot for ATL at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/ATL_permutation_test.png
Observed Spearman correlation for AG: -0.10195307027494525
Permutation test p-value for AG: 0.0001999600079984003
Saved permutation test plot for AG at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/AG_permutation_test.png
Observed Spearman correlation for IFG: -0.022423469738240078
Permutation test p-value for IFG: 0.4081183763247351
Saved permutation test plot for IFG at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/IFG_permutation_test.png
Observed Spearman correlation f